<a href="https://colab.research.google.com/github/Solxcero/_SeSac_/blob/main/1_handwriting_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요라이브러리

In [ ]:
import json
import random
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
%cd /content
!git clone 'https://github.com/Solxcero/-deep-text-recognition-benchmark.git'
%cd /content/-deep-text-recognition-benchmark/

!mkdir data
!mkdir /content/-deep-text-recognition-benchmark/data/train
!mkdir /content/-deep-text-recognition-benchmark/data/test
!mkdir /content/-deep-text-recognition-benchmark/data/validation

!unzip '/content/drive/MyDrive/project/hadwriting/01_handwriting_word_images.zip' -d ./data/


In [ ]:
file = json.load(open('/content/drive/MyDrive/project/hadwriting/handwriting_data_info_clean.json'))
file.keys() #dict_keys(['info', 'images', 'annotations', 'licenses'])
file['info'] 

{'name': '한글 손글씨 데이터', 'date_created': '2019-09-18 04:45:22'}

In [ ]:
words = [f for f in file['annotations'] if f['attributes']['type']=='단어(어절)']
print(f'단어(어절) 데이터 : {len(words)}')


단어(어절) 데이터 : 359997


In [ ]:
# import random
# import os

ocr_words_files = os.listdir('/content/-deep-text-recognition-benchmark/data/1_word/')
# len(ocr_words_files) # 359997

#1000고정
random.seed(1000)
random.shuffle(ocr_words_files)

#8:2 맞추기
n_train = int(len(ocr_words_files) * 0.1)
n_validation = int(len(ocr_words_files) * 0.0125)  #0.25임
n_test = int(len(ocr_words_files) * 0.0125)

print(f'train : {n_train}\nvalidation : {n_validation}\ntest : {n_test}') #
print(f'sum : {n_train+n_validation+n_test}')

train : 35999
validation : 4499
test : 4499
sum : 44997


In [ ]:
# from tqdm import tqdm
%cd /content/-deep-text-recognition-benchmark/data/

train_files = ocr_words_files[:n_train]
validation_files = ocr_words_files[n_train: n_train+n_validation]
test_files = ocr_words_files[-n_test:]

## train/validation/test 이미지들에 해당하는 id 값을 저장

train_img_ids = {}
validation_img_ids = {}
test_img_ids = {}

for image in tqdm(file['images'],desc='id 저장'):
    if image['file_name'] in train_files:
        train_img_ids[image['file_name']] = image['id']
    elif image['file_name'] in validation_files:
        validation_img_ids[image['file_name']] = image['id']
    elif image['file_name'] in test_files:
        test_img_ids[image['file_name']] = image['id']

## train/validation/test 이미지들에 해당하는 annotation 들을 저장

train_annotations = {f:[] for f in train_img_ids.keys()}
validation_annotations = {f:[] for f in validation_img_ids.keys()}
test_annotations = {f:[] for f in test_img_ids.keys()}

train_ids_img = {train_img_ids[id_]:id_ for id_ in train_img_ids}
validation_ids_img = {validation_img_ids[id_]:id_ for id_ in validation_img_ids}
test_ids_img = {test_img_ids[id_]:id_ for id_ in test_img_ids}

for idx, annotation in tqdm(enumerate(file['annotations']),desc='annotation 저장'):
    # if idx % 5000 == 0:
        # print(idx,'/',len(file['annotations']),'processed')
    if annotation['attributes']['type'] != '단어(어절)': #단어 타입이 아니면
        continue
    if annotation['image_id'] in train_ids_img:
        train_annotations[train_ids_img[annotation['image_id']]].append(annotation)
    elif annotation['image_id'] in validation_ids_img:
        validation_annotations[validation_ids_img[annotation['image_id']]].append(annotation)
    elif annotation['image_id'] in test_ids_img:
        test_annotations[test_ids_img[annotation['image_id']]].append(annotation)

with open('train_annotation.json', 'w') as file:
    json.dump(train_annotations, file)
with open('validation_annotation.json', 'w') as file:
    json.dump(validation_annotations, file)
with open('test_annotation.json', 'w') as file:
    json.dump(test_annotations, file)

In [ ]:
# import json
# import os
# import cv2
# import matplotlib.pyplot as plt
# from tqdm import tqdm

## aihub 데이터 annotation을 읽어서 단어 단위로 잘라서 data에 저장하기

data_root_path = '/content/-deep-text-recognition-benchmark/data/1_word/'
save_root_path = '/content/-deep-text-recognition-benchmark/data/'

test_annotations = json.load(open('/content/-deep-text-recognition-benchmark/data/test_annotation.json'))
gt_file = open(save_root_path+'gt_test.txt', 'w')
for file_name in tqdm(test_annotations,desc='test'):
    annotations = test_annotations[file_name]
    image = cv2.imread(data_root_path+file_name)    
    text = annotations[0]['text']      
    cv2.imwrite(save_root_path+'test/'+file_name,image)
    gt_file.write("test/{}\t{}\n".format(file_name, text))

validation_annotations = json.load(open('/content/-deep-text-recognition-benchmark/data/validation_annotation.json'))
gt_file = open(save_root_path+'gt_validation.txt', 'w')
for file_name in tqdm(validation_annotations,desc='validation'):
    annotations = validation_annotations[file_name]
    image = cv2.imread(data_root_path+file_name)    
    text = annotations[0]['text']       
    cv2.imwrite(save_root_path+'validation/'+file_name,image )
    gt_file.write("validation/{}\t{}\n".format(file_name, text))
        
train_annotations = json.load(open('/content/-deep-text-recognition-benchmark/data/train_annotation.json'))
gt_file = open(save_root_path+'gt_train.txt', 'w')
for file_name in tqdm(train_annotations,desc='train'):
    annotations = train_annotations[file_name]
    image = cv2.imread(data_root_path+file_name)   
    text = annotations[0]['text']       
    cv2.imwrite(save_root_path+'train/'+file_name,image)
    gt_file.write("train/{}\t{}\n".format(file_name, text))

train: 100%|██████████| 35999/35999 [01:56<00:00, 310.12it/s]


In [ ]:
!pip install lmdb
!pip install fire
import fire
import cv2
import lmdb,torch,torchvision, nltk ,natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 6.3 MB/s 
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=14a3443cae39315bdf5e8a9b6f5e6fbebbdf3a463a7afb5a7af584e20235088f
  Stored in directory: /root/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built fire


In [ ]:
%cd /content/-deep-text-recognition-benchmark

/content/-deep-text-recognition-benchmark


In [ ]:
!python create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_train.txt --outputPath data_lmdb/train
!python create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_validation.txt --outputPath data_lmdb/validation

!python create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_test.txt --outputPath data_lmdb/test

Written 1000 / 35835
Written 2000 / 35835
Written 3000 / 35835
Written 4000 / 35835
Written 5000 / 35835
Written 6000 / 35835
Written 7000 / 35835
Written 8000 / 35835
Written 9000 / 35835
Written 10000 / 35835
Written 11000 / 35835
Written 12000 / 35835
Written 13000 / 35835
Written 14000 / 35835
Written 15000 / 35835
Written 16000 / 35835
Written 17000 / 35835
Written 18000 / 35835
Written 19000 / 35835
Written 20000 / 35835
Written 21000 / 35835
Written 22000 / 35835
Written 23000 / 35835
Written 24000 / 35835
Written 25000 / 35835
Written 26000 / 35835
Written 27000 / 35835
Written 28000 / 35835
Written 29000 / 35835
Written 30000 / 35835
Written 31000 / 35835
Written 32000 / 35835
Written 33000 / 35835
Written 34000 / 35835
Written 35000 / 35835
Created dataset with 35835 samples
Written 1000 / 4499
Written 2000 / 4499
Written 3000 / 4499
Written 4000 / 4499
Created dataset with 4499 samples
Written 1000 / 4499
Written 2000 / 4499
Written 3000 / 4499
Written 4000 / 4499
Created da

In [ ]:
import shutil
shutil.rmtree('/content/-deep-text-recognition-benchmark/saved_models')

In [ ]:
%%time
!CUDA_VISIBLE_DEVICES=0,1,2 python3 train.py --train_data data_lmdb/train --valid_data data_lmdb/validation \
	--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction CTC \
	--data_filtering_off --workers 0 --imgH 64 --imgW 200

--------------------------------------------------------------------------------
dataset_root: data_lmdb/train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    data_lmdb/train	 dataset: /
sub-directory:	/.	 num samples: 35835
num total samples of /: 35835 x 1.0 (total_data_usage_ratio) = 35835
num samples of / per batch: 64 x 1.0 (batch_ratio) = 64
--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
dataset_root:    data_lmdb/validation	 dataset: /
sub-directory:	/.	 num samples: 4499
--------------------------------------------------------------------------------
model input parameters 64 200 20 1 512 256 1047 25 TPS ResNet BiLSTM CTC
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.Localization

## Test

In [ ]:
! python3 demo.py --image_folder data/test --saved_model /content/-deep-text-recognition-benchmark/saved_models/TPS-ResNet-BiLSTM-CTC-Seed1111/best_accuracy.pth \
    --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction CTC \
    --workers 0 --batch_max_length 50 --imgH 64 --imgW 200

model input parameters 64 200 20 1 512 256 1047 50 TPS ResNet BiLSTM CTC
loading pretrained model from /content/-deep-text-recognition-benchmark/saved_models/TPS-ResNet-BiLSTM-CTC-Seed1111/best_accuracy.pth
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
data/test/00012897.png   	갑                        	0.8917
data/test/00012923.png   	강화하다                     	0.3783
data/test/00013041.png   	그이                       	0.9914
data/test/00013056.png   	수십                       	0.9929
data/test/00013071.png   	수집하다                     	0.7322
data/test/00013252.png   	훈전                       	0.3031
data/test/00013302.png   	부드럽다                     	0.9030
data/test/00013385.png   	나오다                      	0.9646
data/test/00013531.png   	둘                        	0.8350
data/test/00013541.png   	뒤지다  

In [ ]:
! python3 demo.py --image_folder data/test --saved_model /content/-deep-text-recognition-benchmark/saved_models/TPS-ResNet-BiLSTM-CTC-Seed1111/best_accuracy.pth \
    --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction CTC \
    --workers 0 --batch_max_length 50 --imgH 64 --imgW 200

model input parameters 64 200 20 1 512 256 1047 50 TPS ResNet BiLSTM CTC
loading pretrained model from /content/-deep-text-recognition-benchmark/saved_models/TPS-ResNet-BiLSTM-CTC-Seed1111/best_accuracy.pth
Figure(640x480)
data/test/00012897.png   	갑                        	0.8917
Figure(640x480)
data/test/00012923.png   	강화하다                     	0.3783
Figure(640x480)
data/test/00013041.png   	그이                       	0.9914
Figure(640x480)
data/test/00013056.png   	수십                       	0.9929
Figure(640x480)
data/test/00013071.png   	수집하다                     	0.7322
Figure(640x480)
data/test/00013252.png   	훈전                       	0.3031
Figure(640x480)
data/test/00013302.png   	부드럽다                     	0.9030
Figure(640x480)
data/test/00013385.png   	나오다                      	0.9646
Figure(640x480)
data/test/00013531.png   	둘                        	0.8350
Figure(640x480)
data/test/00013541.png   	뒤지다                      	0.8041
Figure(640x480)
data/test/00017590.png   	시

In [ ]:
# !zip 해서 content 밑에 압출해두기 

In [ ]:
# !zip -r /content/drive/MyDrive/project/test01.zip /content/-deep-text-recognition-benchmark



zip error: Interrupted (aborting)
